In [10]:
import csv 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
RND_MEAN = 0
RND_STD = 1.0
LEARNING_RATE = 0.001

# method definition
def main(epoch_count=30, mb_size=30, report=1, train_ratio=0.8):
    load_dataset()
    init_param()
    train_and_test(epoch_count, mb_size, report, train_ratio)



# method definition
def load_dataset():
    with open('Regression_data.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        rows = []
        for row in csvreader:
            rows.append(row)
    
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    
    data = np.zeros([len(rows), input_cnt + output_cnt])
    for n, row in enumerate(rows):
        if row[0] == 'M':
            data[n, 0] = 1
        if row[0] == 'F':
            data[n, 1] = 1
        if row[0] == 'I':
            data[n, 2] = 1
        data[n, 3:] = row[1:]

    # Data normalization
    # data[:, 3:] = (data[:, 3:] - np.mean(data[:, 3:], axis=0)) / np.std(data[:, 3:], axis=0)


# method definition
def init_param():
    global weight, bias
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

# method definition
def forward_neuralnet(x):
    y_hat = np.matmul(x, weight) + bias
    print("y_hat: ", y_hat)
    return y_hat, x

# method definition
def forward_postproc(y_hat, y):
    diff = y_hat - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

# method definition
def eval_accuracy(y_hat, y):
    mdiff = np.mean(np.abs((y_hat - y) / y))
    return 1 - mdiff

def backprop_neuralnet(G_output, x):
    global weight, bias
    x_transpose = x.transpose()
    G_w = np.matmul(x_transpose, G_output)
    G_b = np.sum(G_output, axis=0)
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

def backprop_postproc(diff):
    M_N = diff.shape
    g_mse_square = np.ones(M_N) / np.prod(M_N)
    g_square_diff = 2 * diff
    g_diff_output = 1
    G_diff = g_mse_square * g_square_diff
    G_output = g_diff_output * G_diff
    return G_output

# Override method
def run_train(x, y):
    
    y_hat, aux_nn_x = forward_neuralnet(x)
    print(y_hat)
    loss, aux_pp_diff = forward_postproc(y_hat, y)
    accuracy = eval_accuracy(y_hat, y)
    G_output = backprop_postproc(aux_pp_diff)
    backprop_neuralnet(G_output, aux_nn_x)
    return loss, accuracy

# Override method
def run_test(x, y):
    
    y_hat, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(y_hat, y)
    return accuracy

# New method definition
def batch_normalization(x):
    mean = np.mean(x, axis=0)
    std = np

In [ ]:
# 메서드 정의 
def arrange_data(mb_size, train_ratio):
    
    # 활용 빈도가 높은 변수들은 전역변수로 할당하며, 활용빈도가 낮은 변수의 경우 반환처리 하였습니다. 
    global data, shuffle_map, test_begin_index

    '''세부 기능 1)전체 데이터의 인덱싱 생성 후 셔플링.'''
    # 실험에 쓰일 데이터(data)의 수에 맞게 인덱싱(0,1,2,3,...)을 생성하는 과정 입니다.
    # 입출력 예시
    # >>>data.shape[0] 
    # 20
    shuffle_map = np.arange(data.shape[0])
    # 학습의 효율을 높여주기 위해 인덱싱(shuffle_map)을 뒤섞어 줍니다.
    np.random.shuffle(shuffle_map)

    '''세부 기능 2)1에폭에 필요한 미니배치의 수 연산.'''
    # 1 에폭을 위한 미니배치의 수(mini_batch_step_count)는 
    # 전체 데이터(data.shape[0])에서 학습 데이터의 비율(train_ratio)만큼의 개수를 구한 후 
    # 미니배치 크기(mb_size)로 나눠 몫 만을 구해줍니다. 
    # ( ※ mb_size 는 하나의 미니배치에 포함된 데이터의 수 입니다.)
    mini_batch_step_count = int(data.shape[0] * train_ratio) // mb_size

    '''세부 기능 3)학습 데이터와 테스트 데이터의 경계 인덱스를 구함.'''
    # 전체 데이터에서 학습 데이터와 테스트 데이터가 나뉘는 경계가 되는 인덱스를 
    # 테스트 데이터가 시작되는 인덱스로 설정하였습니다. 
    # 이 값을 구하기 위해서는 다양한 방식이 있지만, 
    # 미니배치 스탭 수(mini_batch_step_count)와 미니배치 크기(mb_size)를 곱해 
    # 학습 및 테스트 데이터 경계 인덱스(test_begin_index)를 구해줍니다.
    test_begin_index = mini_batch_step_count * mb_size
    

    # 다수의 변수가 생성되었지만 mini_batch_step_count 변수만 밖으로 반환합니다.
    # 활용 빈도가 높은 변수들은 전역변수로 할당하며, 활용빈도가 낮은 변수의 경우 반환처리 하였습니다. 
    return mini_batch_step_count

In [6]:
# 메서드 정의
def get_train_data(mb_size, nth):
    
    # 학습 데이터의 경우 새로운 Epoch 의 학습이 수행되기 전(if nth == 0)
    # 학습 데이터 인덱싱 정보를 무작위로 섞어(np.random.shuffle()) 학습의 효과를 높여줍니다.
    # 학습의 효과를 높여주는 선행 연구에 근거함
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_index])

    # 학습 데이터의 분리는 미니배치를 고려해야 합니다. 
    # 전체 데이터(data)에서 뒤섞인 인덱싱값으로 접근(shuffle_map)하여 미니배치 처리를 수행합니다.
    # 만약 미니배치 사이즈(mb_size) 값이 4 라면 다음과 같이 나눠볼 수 있습니다. 
    # 0 : 4
    # 4 : 8
    # 8 : 12
    # 12 : 16
    # 그리고 이와 같은 배수의 연산을 위한 수식을 미니배치 크기(mb_size)와 몇 번째 미니배치 처리 단계(nth) 값을 활용합니다.
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth+1)]]
    
    # 미니배치에 따라 나눠진 학습 데이터를 입출력 벡터로 나눠 반환합니다. 
    return train_data[:,:-output_cnt], train_data[:,-output_cnt:]

In [7]:
# 메서드 정의
def get_test_data():

    """ 전체 데이터에서 테스트 데이터 분리 과정 """
    # 전체 데이터(data)의 인덱싱 정보를 갖고 있는 변수(shuffle_map)에 접근하여 테스트 데이터를 얻어냅니다. 
    # 테스트 데이터의 경우 테스트 데이터의 시작 인덱싱 위치(test_begin_index)를 기준으로 인덱싱 끝 까지 범위를 갖도록 합니다. 
    test_data = data[shuffle_map[test_begin_index:]]

    """ 테스트 데이터에서 입출력 벡터 분리 과정 """
    # test_data 변수는 입출력 벡터를 포함한 테스트 데이터 입니다. 
    # 그렇기에 입출력 벡터를 분리하기 위한 과정을 수행하여야 합니다. 
    # 출력 계층 값을 담는 변수(output_cnt)를 활용하여 슬라이싱 과정을 수행합니다. 
    # 행은 모두 포함하지만 열의 경우 다음과 같이 구분할 수 있습니다.  
    return test_data[:,:-output_cnt], test_data[:,-output_cnt:]

In [8]:
# 메서드 정의 
def train_and_test(epoch_count, mb_size, report, train_ratio):
    
    '''1) 데이터 셔플링 - arrange_data()'''
    # 메서드를 통해 데이터를 섞어주고 1 에폭에 필요한 미니배치 수 를 반환합니다. 
    mini_batch_step_count = arrange_data(mb_size,train_ratio)

    '''2) 테스트 데이터 분리 - get_test_data()'''
    # 학습 데이터는 미니배치를 고려하여야 하기에 
    # 미니배치 고려가 필요 없는 테스트 데이터 분리 메서드를 먼저 배치합니다. 
    test_x, test_y = get_test_data()

    # 다음은 학습 및 테스트 수행 단계를 정의하는 과정 입니다.
    # 학습은 사용자가 지정한 학습횟수(에폭-epoch_count)에 맞춰 학습을 반복하도록 정의합니다. 
    for epoch in range(epoch_count):

        # 학습 진행에 따른 정보를 담기 위해 빈 리스트를 사전에 정의하여 줍니다. 
        losses, accs = [], []

        # 1 에폭이 수행되기 위해서는 미니배치 처리에 따른 학습이 모두 완료되어야 합니다. 
        # 그리고 에폭은 사용자가 1 이상의 값을 설정할 수 있어야 하므로, 
        # 이를 위한 이중 반복문 구조를 설계할 수 있습니다. 
        # 만약 학습 데이터의 수가 16개, 미니배치 크기(mb_size)가 4개라면 
        # 1 에폭을 위해서는 총 4번(mini_batch_step_count)의 미니배치에 따른 학습이 수행되어야 합니다. 
        for nth in range(mini_batch_step_count):
            
            '''3) 학습 데이터 분리 - get_train_data()'''
            # 학습 데이터의 경우 에폭마다의 미니배치 처리 단계(nth)를 고려하여 학습 데이터의 입출력 벡터를 변수화 합니다. 
            train_x, train_y = get_train_data(mb_size, nth)
            
            '''4) 학습 - run_train()'''
            # 미니배치 처리 단계(nth)에 따른 학습 데이터의 입출력 벡터를 학습합니다.
            # 지금은 run_train() 메서드의 이름만 정의해둔 상황입니다. 
            # 정리하면 미니배치 단계에 따라 임의로 설정한 고정된 실험 결과값을 반환합니다. 
            #TODO run_train() 메서드는 다음 과정에서 하위 메서드 정의
            loss, acc = run_train(train_x,train_y)
            
            # 미니배치 단계에 따른 실험 결괏값(loss, acc)들을 append() 메서드로 묶어줍니다. 
            losses.append(loss)
            accs.append(acc)

        '''5) 테스트 - run_test()'''
        # 테스트 과정은 사용자가 지정한 1 이상의 결과 보고 주기(report)에 따른 에폭 마다 수행하도록 합니다. 
        # 모든 에폭마다 테스트를 수행할 수도 있지만, 그건 효율적인 방식이 되지 못합니다. 
        
        # 상황 예제 1) 
        # 만약 학습 반복 주기(에폭)를 10 이라 하며, 결과 보고 주기(report)를 2 라 하면 
        # 다음과 같은 주기로 테스트 결과를 반환하도록 합니다.
        # 0(x), 1(o), 2(x), 3(o), 4(x), 5(o), 6(x), 7(o), 8(x), 9(o) 
        
        # 상황 예제 2) 
        # 만약 학습 반복 주기(에폭)를 10 이라 하며, 결과 보고 주기(report)를 3 이라 하면 
        # 다음과 같은 주기로 테스트 결과를 반환하도록 합니다.
        # 0(x), 1(x), 2(o), 3(x), 4(x), 5(o), 6(x), 7(x), 8(o), 9(x) 
        
        # 이와같은 조건을 갖는 주기를 반환할 수 있도록 코드를 작성합니다. 

        # 이러한 경우 나머지를 반환하는 연산자와 비교 연산자를 활용할 수 있습니다. 
        if report > 0 and (epoch+1) % report == 0:
            
            # 위 조건에 부합하는 경우 테스트를 진행합니다. 
            acc = run_test(test_x, test_y)

            # 수행한 결과를 사용자에게 출력합니다. 
            # 현재 에폭
            # 현재 에폭에 따른 미니배치 단계별 결과의 학습 손실(loss) 평균값
            # 현재 에폭에 따른 미니배치 단계별 결과의 학습 정확도(accs) 평균값
            # 현재 에폭의 정확도(loss)
            print("Epoch {}   : Train - Loss = {:.3f}, Accuracy = {:.3f} / Test - Accuracy = {:.3f}".\
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
        
    '''5) 최종 테스트 - run_test()'''
    # 학습을 모두 마쳤다면 주어진 조건에 다른 모델 파라미터에 대한 조정이 완료되었다는 의미 입니다. 
    # 그렇기에 최종 테스트를 수행하여 학습을 모두 마친 AI 모델에 대한 성능 평가를 run_test() 메서드로 수행합니다. 
    final_acc = run_test(test_x, test_y)

    # 학습에 따른 최종 결괏값 출력
    print("="*30, ' Final TEST ', '='*30)
    print('\nFinal Accuracy = {:.3f}'.format(final_acc))